In [ ]:
import os
import gmsh
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from fem_pre_processing import read_mesh
from fem_processing import vectorial_matrices_assembly as assembly
from fem_processing import master_domain as master
from fem_pos_processing import graph_results as graph

# `get_edge_key_map()`

In [ ]:
def get_edge_key_map(mesh_data):
    # Construir um dicionário auxiliar para mapear arestas diretamente às suas chaves
    edge_key_map = {}
    for key, edge in mesh_data['edges'].items():
        # Usamos uma tupla ordenada para representar a aresta, garantindo unicidade independente da direção
        edge_nodes = tuple(sorted(edge['conn']))
        edge_key_map[edge_nodes] = key
    return edge_key_map

# `apply_physics()`

In [2]:
def apply_physics(mesh_data):
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']
    edges_data = mesh_data['edges']
    
    # Mapeamento das arestas diretamente às suas chaves    
    edge_key_map = get_edge_key_map(mesh_data)

    for cell in cell_data.values():  
        # Atualiza a conectividade de arestas da célula
        cell['conn_edge'] = [
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][1]]))],  # e1: 1 -> 2
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][2]]))],  # e2: 1 -> 3
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][3]]))],  # e2: 1 -> 4
            edge_key_map[tuple(sorted([cell['conn_sorted'][1], cell['conn_sorted'][2]]))],  # e3: 2 -> 3
            edge_key_map[tuple(sorted([cell['conn_sorted'][1], cell['conn_sorted'][3]]))],  # e3: 2 -> 4
            edge_key_map[tuple(sorted([cell['conn_sorted'][2], cell['conn_sorted'][3]]))],  # e3: 3 -> 4
        ]

        # Geometric data    
        xc = np.mean([mesh_data['nodes'][node]['xg'][0] for node in cell['conn']])
        yc = np.mean([mesh_data['nodes'][node]['xg'][1] for node in cell['conn']])
        zc = np.mean([mesh_data['nodes'][node]['xg'][2] for node in cell['conn']])
        Je = assembly.jacobian_tetrahedra(mesh_data, cell)
        
        # Propriedades geométricas da célula
        cell['geo']['centroid'] = (xc, yc, zc)
        cell['geo']['jacobian'] = Je
        cell['geo']['dim'] = np.abs(np.linalg.det(Je)) * 0.5 

        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['p(x)'] = 1
        cell['q(x)'] = np.eye(3)
        cell['f(x)'] = np.zeros((3, 1))

        # Inicializa os contornos do domínio
        gamma_d = {}
        for idx, node in enumerate(cell['conn']):
            bc_type = nodes_data[node]['bc']['type']

            if bc_type == 'Dirichlet':
                gamma_d[idx] = node
                # nodes_data[node]['bc']['value'] = 0   # Dirichlet homogêneo

        # Define o contorno do elemento, caso aplicável
        if gamma_d:
            cell['contour'] = {'type': 'gamma_d', 'conn_dict': gamma_d}

        # Atualiza dados físicos das arestas
        for edge in edges_data.values():
            # Nós da aresta
            n0, n1 = edge['conn'][0], edge['conn'][1]
            
            # Adiciona os potenciais de Dirichlet sobre as arestas
            if nodes_data[n0]['bc']['type'] == 'Dirichlet' and nodes_data[n1]['bc']['type'] == 'Dirichlet':
                edge['bc'] = {
                    'tag': 101,
                    'type': 'Dirichlet',
                    'value': nodes_data[n0]['bc']['value'],
                    'name': 'contour_domain'}
                
            # Coordenadas dos nós inicial e final
            x0, y0, z0 = mesh_data['nodes'][n0]['xg']
            x1, y1, z1 = mesh_data['nodes'][n1]['xg']
        
            # Adiciona o tamanho da aresta
            edge['len'] = ((x1 - x0)**2 + (y1 - y0)**2 + (z1 - z0)**2) ** 0.5
    
    return mesh_data

# `create_rectangular_guide()`

In [ ]:
def create_rectangular_cavity(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT
    problem_dim = 3

    # Dimensões da cavidade retangular, metros
    # a, b, c = 1.0, 0.6, 0.4
    a, b, c = 1.0, 1.0 , 1.0

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_cavity")

    # Criar volume retangular (caixa)
    TagVolume = gmsh.model.occ.addBox(0, 0, 0, a, b, c)
    gmsh.model.occ.synchronize()

    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)

    gmsh.model.mesh.generate(problem_dim)
    gmsh.model.mesh.setOrder(order)

    # Obter os contornos (superfícies, dim=2) do volume
    BoundaryDimTags = gmsh.model.getBoundary([(3, TagVolume)], oriented=True, recursive=False)

    # Exibir os TAGs das superfícies associadas a cada contorno
    BoundaryTags = [Dimtags[1] for Dimtags in BoundaryDimTags]

    # Definindo as superfícies de contorno de Dirichlet (dim=2)
    gmsh.model.addPhysicalGroup(dim=2, tags=BoundaryTags, tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

    # Adicionar grupos físicos para Dim=3 (volume)
    gmsh.model.addPhysicalGroup(dim=3, tags=[TagVolume], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_cavity_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)

    # Create mesh Structure Data from gmsh
    read_mesh.basic_info(problem_dim)
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL, problem_dim)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY, problem_dim-1)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Finalize the gmsh model e return the mesh data
    gmsh.finalize()
    return mesh_data

# `waveguide_modes()`

In [ ]:
def waveguide_modes(mn_max, mode_type):
    """
    Calcula os modos TE ou TM para um guia de onda retangular.

    Parâmetros:
    a (float): Dimensão transversal a (em metros).
    b (float): Dimensão transversal b (em metros).
    m_max (int): Número máximo de modos para m.
    n_max (int): Número máximo de modos para n.
    mode_type (str): Tipo de modo, 'TE' ou 'TM'.

    Retorna:
    dict: Dicionário com os modos e seus respectivos kc.
    """
    # Dimensões do guia de onda retangular
    a, b = 8e-2, 4e-2 

    if mode_type not in ['TE', 'TM']:
        raise ValueError("mode_type deve ser 'TE' ou 'TM'")

    modes = {}
    for m in range(mn_max + 1):
        for n in range(mn_max + 1):
            if mode_type == 'TE' and m == 0 and n == 0:
                continue  # Excluir o modo (0,0) para TE
            if mode_type == 'TM' and (m == 0 or n == 0):
                continue  # Excluir modos onde m=0 ou n=0 para TM
            kc = np.sqrt((m * np.pi / a)**2 + (n * np.pi / b)**2)
            modes[f"{mode_type}_{m}{n}"] = kc**2

    # Ordenar os autovalores e listar os cinco menores
    sorted_te_modes = sorted(modes.items(), key=lambda x: x[1])
    smallest_modes = {key: value 
                      for i, (key, value) in enumerate(sorted_te_modes) if i < mn_max}

    # Imprimir os menores autovalores
    print(f"\n{mn_max} menores autovalores analíticos dos modos {mode_type}:")
    for mode, kc2 in smallest_modes.items():
        print(f"{mode}: kc2 = {kc2:.3f} rad2/m2")
    
    return smallest_modes

# `calculate_error()`

In [2]:
def calculate_error(modes_u, modes_uh):
    """
    Calcula o erro entre os valores de referência (modos analíticos) e os modos computados numericamente.

    Parâmetros:
    reference_modes (list): Lista de tuplas representando os modos analíticos, onde cada tupla contém o nome do modo e o valor de kc.
    computed_modes (list): Lista de valores computados numericamente para os menores kc^2.

    Retorna:
    dict: Um dicionário contendo os erros absolutos para cada modo correspondente.
    """
    # Compatibilizar tamanhos
    smallest_modes_uh = modes_uh[:len(modes_u)]
    
    # Calcular os erros percentuais
    errors = [abs(ref - comp)/ref * 100
               for ref, comp in zip(modes_u.values(), smallest_modes_uh)]

    # Retornar os erros em um dicionário
    errors_dict = {mode: error for mode, error in zip(modes_u.keys(), errors)}

    # Imprimir dicionário de erros
    for mode, error in errors_dict.items():
        print(f"Modo {mode}: Erro percentual = {error:.4f}")

    return errors_dict

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.